# Chapter 13. Naive bayes

It is well for the heart to be naive and for the mind not to be - Anatole France

Example: Spam filtering for the message feature

## 13.1 A really dumb spam filter

S = event the message is spam 

B = event the message contains the word bitcoin

P(S|B) = [P(B|S)P(S)]/[P(B|S)P(S) + P(B|¬S)P(¬S)]

If we have a large collection of messages we know are spam, and a large collection of messages we know are not spam, then we can easily estimate. 

For example, assume that any message is equally likely to be spam or not spam so P(S) = P(¬S) = 0.5. If 50% of spam messages have the word bitcoin, but only 1% of non-spam messages do, then the probability that any given bitsoin-containing email is spam is: 0.5 / (0.5 + 0.01) = 98%

## 13.2 A more sophisticated spam filter

For many words, Xi = event a message contains the word wi

The key to naive bayes is making the big assumption that the presences or absences of each word are independent of one another, conditional on a message being spam or not. Intuitively, this assumption means that knowing whether a certain spam message contains the word bitcoin gives you no information about whether that same message contains the word rolex

The naive bayes assumption allows us to compute each of the probabilities on the right simply by multiplying together the individual probability estimates for each vocabulary word. In practice, to prevent the problem of underflow, in which computers don't deal well with floating-point numbers that are too close to 0, we usually compute the equivalent exp(log(p1) + ... log(pn))

To estimate the P(Xi|S) and P(Xi|¬S), the probabilities that a spam message or nonspam message contains the word wi. If we have a fair number of training messages labeled as spam and not spam, we can estimate the P(Xi|S) simply as the fraction of spam messages containing the word wi. However, this causes a big problem. Imagine that in our training set the vocabulary word data only occurs in nonspam messages, then we estimate P(data|S) = 0. The result is that the naive bayes classifier would always assign spam probability 0 to any message containing the word data, even a message like "data on free bitsoin and authentic rolex watches"/ To avoid this problem, we usually use some kind of smoothing. We will choose a pseudocount k and estimate the probability of seeing the ith word in a spam message as: P (Xi|S) = (k + number of spams containing wi) / (2k + number of spams). We do similarly for P(Xi|¬S). That is, when computing the spam probabilities for the ith word, we assume we also saw k additional nonspams containing the word and k additional nonspams not containing the word.

For example, if data occurs in 0/98 spam messages, and if k is 1, we estimate P(data|S) as 1/100 = 0.01, which allows our classifier to still assign some nonzero spam probability to messages that contain the word data.

## 13.3 Implementation

In [1]:
from typing import Set
import re

def tokenize(text: str) -> Set[str]:
    text = text.lower() # convert to lowercase
    all_words = re.findall('[a-z0-9]+', text) # extract the words
    return set(all_words) # remove duplicates

assert tokenize('Data Science is science') == {'data', 'science', 'is'}

As the classifier needs to keep track of tokens, counts, and labels from the training data, we'll make it a class

In [4]:
from typing import NamedTuple 

class Message(NamedTuple):
    text: str
    is_spam: bool

In [6]:
from typing import List, Tuple, Dict, Iterable 
import math 
from collections import defaultdict 

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k # smoothing factor 
        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0
    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts 
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1
            
            # Increment word counts 
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1
    def _probabilities(self, token: str) -> Tuple[float, float]:
        '''returns P(token / spam) and P(token / ham)'''
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]
        
        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)
        
        return p_token_spam, p_token_ham 
    
    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0
        
        # Iterate through each word in our vocabulary
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)
            
            # If token appears in the message, add the log probability of seeing it 
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)
            
            # Otherwise add the log probability of not seeing it which is log(1 - probability of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)
        
        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

## 13.4 Testing our model

In [8]:
messages = [Message('spam rules', is_spam = True),
            Message('ham rules', is_spam = False),
            Message('hello ham', is_spam = False)]
model = NaiveBayesClassifier(k = 0.5)
model.train(messages)

In [9]:
assert model.tokens == {'spam', 'ham', 'rules', 'hello'}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {'spam': 1, 'rules': 1}
assert model.token_ham_counts == {'ham': 2, 'rules': 1, 'hello': 1}

In [10]:
text = 'hello spam'

probs_if_spam = [
   (1 + 0.5) / (1 + 2 * 0.5), # "spam" (present)
   1 - (0 + 0.5) / (1 + 2 * 0.5), # "ham" (not present)
   1 - (1 + 0.5) / (1 + 2 * 0.5), # "rules" (not present)
   (0 + 0.5) / (1 + 2 * 0.5) # "hello" (present)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5), # "spam" (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5), # "ham" (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5), # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5), # "hello" (present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

## 13.5 Using our model

In [11]:
from io import BytesIO
import requests 
import tarfile 

BASE_URL = "https://spamassassin.apache.org/old/publiccorpus"
FILES = ["20021010_easy_ham.tar.bz2",
         "20021010_hard_ham.tar.bz2",
         "20021010_spam.tar.bz2"]

OUTPUT_DIR = 'spam_data'

for filename in FILES:
    # Use requests to get the file contents at each URL.
    content = requests.get(f"{BASE_URL}/{filename}").content
    # Wrap the in-memory bytes so we can use them as a "file."
    fin = BytesIO(content)
    # And extract all the files to the specified output dir.
    with tarfile.open(fileobj=fin, mode='r:bz2') as tf:
        tf.extractall(OUTPUT_DIR)

In [ ]:
import glob, re

# modify the path to wherever you've put the files
path = 'spam_data/*/*'

# glob.glob returns every filename that matches the wildcarded path
for filename in glob.glob(path):
    is_spam = "ham" not in filename 

    # There are some garbage characters in the emails; the errors='ignore'
    # skips them instead of raising an exception.
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith("Subject:"):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break # done with this file

In [ ]:
import random
from typing import TypeVar, List, Tuple
X = TypeVar('X')  # generic type to represent a data point

def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """Split data into fractions [prob, 1 - prob]"""
    data = data[:]                    # Make a shallow copy
    random.shuffle(data)              # because shuffle modifies the list.
    cut = int(len(data) * prob)       # Use prob to find a cutoff
    return data[:cut], data[cut:]     # and split the shuffled list there.

random.seed(0) 
train_messages, test_messages = split_data(data, 0.75)
model = NaiveBayesClassifier()
model.train(train_messages)

In [ ]:
from collections import Counter

predictions = [(message, model.predict(message.text))
                for message in test_messages]

# Assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
confusion_matrix = Counter((message.is_spam, spam_probability > 0.5)
                           for message, spam_probability in predictions)

print(confusion_matrix)

In [ ]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
    # We probably shouldn't call private methods, but it's for a good cause.
    prob_if_spam, prob_if_ham = model._probabilities(token)
    return prob_if_spam / (prob_if_spam + prob_if_ham)

words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))

print("spammiest_words", words[-10:])
print("hammiest_words", words[:10])

**Improve the model performance**:
1. Look at the message content, not just the subject line. You’ll have to be careful how you deal with the message headers.
2. Our classifier takes into account every word that appears in the training set, even words that appear only once. Modify the classifier to accept an optional min_count threshold and ignore tokens that don’t appear at least that many times
3. The tokenizer has no notion of similar words (e.g., cheap and cheapest). Modify the classifier to take an optional stemmer function that converts words to equivalence classes of words. For example, a really simple stemmer function might be. Creating a good stemmer function is hard. People frequently use the Porter stemmer.
4. Although our features are all of the form “message contains word wi,” there’s no reason why this has to be the case. In our implementation, we could add extra features like “message contains a number” by creating phony tokens like contains:number and modifying the tokenizer to emit them when appropriate

In [ ]:
def drop_final_s(word):
    return re.sub("s$", "", word)

## 13.6 For further exploration

scikit-learn BernoulliNB

In [ ]:
from sklearn.naive_bayes import BernoulliNB